In [6]:
import numpy as np
import glob
from matplotlib import pyplot as plt
from PIL import Image
import random
import os
from tqdm import tqdm
import cv2

In [11]:
# Baseline mask와 BinarySegmentationModel 합치기
src_BSM = './BinarySegmentationModel/res/'
src_baseline= './BaselineModel/results/pred/SoTa_20221115_021934/mask/'
save_path = './res/aggregated_res/'
origin_file_path = './test/x/'
image_type = '*.png'
BSM_paths = glob.glob(src_BSM+image_type)
baseline_paths = glob.glob(src_baseline+image_type)

for path in tqdm(BSM_paths):
    filename = os.path.basename(path)
    BSM_image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    baseline_image = cv2.imread(src_baseline+filename,cv2.IMREAD_GRAYSCALE)

    origin_size = cv2.imread(origin_file_path+filename).shape[0]
    BSM_image = cv2.resize(BSM_image, [origin_size, origin_size], interpolation=cv2.INTER_NEAREST)

    baseline_image_left = baseline_image[:,:origin_size]
    baseline_image_right = baseline_image[:,origin_size:]

    BSM_image_left = np.array(BSM_image)
    BSM_image_right = np.array(BSM_image)
    cnt, labels = cv2.connectedComponents(BSM_image_left) #754 754
    
    #left
    for i in range(1,cnt):
        tmp_right = baseline_image_right[labels == i]
        in1 = 1 in tmp_right
        in3 = 3 in tmp_right
        tmp_left = baseline_image_left[labels == i]
        in2 = 2 in tmp_left

        if in1 + in2 + in3 > 1:
            now = [(tmp_right == 1).sum(),(tmp_left == 2).sum(),(tmp_right == 3).sum()]
            idx = now.index(max(now)) + 1

            if(idx in [2]):
                BSM_image_left[labels == i] = idx
            else:
                BSM_image_left[labels == i] = 0

        else:
            if in2:
                BSM_image_left[labels == i] = 2
            else:
                BSM_image_left[labels == i] = 0

    # right
    for i in range(1,cnt):
        tmp_right = baseline_image_right[labels == i]
        in1 = 1 in tmp_right
        in3 = 3 in tmp_right
        tmp_left = baseline_image_left[labels == i]
        in2 = 2 in tmp_left

        if in1 + in2 + in3 > 1:
            now = [(tmp_right == 1).sum(),(tmp_left == 2).sum(),(tmp_right == 3).sum()]
            idx = now.index(max(now)) + 1
        
            if(idx in [1,3]):
                BSM_image_right[labels == i] = idx
            else:
                BSM_image_right[labels == i] = 0

        else:
            if in1:
                BSM_image_right[labels == i] = 1
            elif in2:
                BSM_image_right[labels == i] = 0
            elif in3:
                BSM_image_right[labels == i] = 3
            else:
                BSM_image_right[labels == i] = 3

    BSM_image_final = np.concatenate((BSM_image_left,BSM_image_right),axis=1)
    cv2.imwrite(save_path + filename,BSM_image_final)

100%|██████████| 2338/2338 [01:39<00:00, 23.43it/s]


In [12]:
# 일정 개수 이하의 라벨은 모두 제거하기
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm

save_path = './res/aggregated_res/'
save_to_path = './res/final_res/'
file_type = '*.png'
paths = glob.glob(save_path + file_type)
change_list =[]
for path in tqdm(paths):
    filename = os.path.basename(path)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = np.array(img)
    thread_hold = 30
    for j in range(3):
        idx =j+1
        if (sum(img[img==idx]) < thread_hold):
            img[img==idx] = 0

    result = Image.fromarray(img.astype(np.uint8))
    result.save(save_to_path + filename)

100%|██████████| 2338/2338 [00:46<00:00, 50.34it/s]


In [13]:
# rgb로 결과 확인하기
final_res_path = './res/final_res/*.png'
rgb_final_res_path = './res/final_rgb/'
paths = glob.glob(final_res_path)
for path in tqdm(paths):
    filename = os.path.basename(path)
    img = np.array(Image.open(path))
    img_dim_2 = img
    img = np.expand_dims(img_dim_2,axis=0)
    zeros = np.concatenate((np.zeros_like(img),np.zeros_like(img),np.zeros_like(img)),axis=0)
    for i in range(3):
        idx = i+1
        zeros[i][img_dim_2==idx] = 255

    res = np.transpose(zeros,(1,2,0))
    res = Image.fromarray(res.astype(np.uint8))
    res.save(rgb_final_res_path + filename)
    

100%|██████████| 2338/2338 [01:57<00:00, 19.92it/s]
